In [38]:
from pathlib import Path
import duckdb
from IPython.core.magic import register_cell_magic
import pandas as pd

pd.set_option("display.max_rows", 20)       # show all rows
pd.set_option("display.max_columns", None)    # show all columns
pd.set_option("display.max_colwidth", None)   # don't truncate text
pd.set_option("display.expand_frame_repr", False)  # don't wrap across lines


# 1) Where to store the DuckDB database file (change if you want it elsewhere)
DB_PATH = Path("./plt.duckdb")
DB_PATH.parent.mkdir(parents=True, exist_ok=True)
# print(DB_PATH)
con = duckdb.connect(str(DB_PATH))

@register_cell_magic
def ducksql(line, cell):
    res = con.sql(cell)
    try:
        display(res.df())
    except Exception:
        print("OK")

IOException: IO Error: Could not set lock on file "/Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/python_scripts/plt.duckdb": Conflicting lock is held in /Users/mohammed/.pyenv/versions/3.12.6/bin/python3.12 (PID 40416) by user mohammed. See also https://duckdb.org/docs/connect/concurrency

In [34]:
%%ducksql
SELECT * 
FROM analytics_staging.stg_transfer_data 
LIMIT 50;

CatalogException: Catalog Error: Table with name stg_transfer_data does not exist!
Did you mean "plt.pg_catalog.pg_attrdef"?

In [2]:
%%ducksql
CREATE SCHEMA IF NOT EXISTS staging;

-- 2) Point directly at your file (absolute path)
CREATE OR REPLACE VIEW staging.transfers_all AS
SELECT *
FROM read_csv_auto('/Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/dbt/seeds/premier-league.csv', HEADER=TRUE);

SELECT * FROM staging.transfers_all LIMIT 50;

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season
0,Middlesbrough FC,Tommy Wright,26,Left Winger,Leicester,€910Th.,in,Summer,0.91,Premier League,1992,1992/1993
1,Middlesbrough FC,Jonathan Gittens,28,defence,Southampton,€250Th.,in,Summer,0.25,Premier League,1992,1992/1993
2,Middlesbrough FC,Chris Morris,28,Right-Back,Celtic,?,in,Summer,NA,Premier League,1992,1992/1993
3,Middlesbrough FC,Ben Roberts,17,Goalkeeper,Boro U18,-,in,Summer,NA,Premier League,1992,1992/1993
4,Middlesbrough FC,Andy Todd,17,Centre-Back,Boro U18,-,in,Summer,NA,Premier League,1992,1992/1993
...,...,...,...,...,...,...,...,...,...,...,...,...
45,Sheffield Wednesday,Paul Williams,27,attack,Crystal Palace,?,out,Summer,NA,Premier League,1992,1992/1993
46,Wimbledon FC,Dean Holdsworth,23,Centre-Forward,Brentford,€1.00m,in,Summer,1,Premier League,1992,1992/1993
47,Wimbledon FC,Vinnie Jones,27,Defensive Midfield,Chelsea,€830Th.,in,Summer,0.83,Premier League,1992,1992/1993
48,Wimbledon FC,Greg Berry,21,midfield,Leyton Orient,€313Th.,in,Summer,0.313,Premier League,1992,1992/1993


In [3]:
%%ducksql

SELECT * FROM staging.transfers_all LIMIT 50

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season
0,Middlesbrough FC,Tommy Wright,26,Left Winger,Leicester,€910Th.,in,Summer,0.91,Premier League,1992,1992/1993
1,Middlesbrough FC,Jonathan Gittens,28,defence,Southampton,€250Th.,in,Summer,0.25,Premier League,1992,1992/1993
2,Middlesbrough FC,Chris Morris,28,Right-Back,Celtic,?,in,Summer,NA,Premier League,1992,1992/1993
3,Middlesbrough FC,Ben Roberts,17,Goalkeeper,Boro U18,-,in,Summer,NA,Premier League,1992,1992/1993
4,Middlesbrough FC,Andy Todd,17,Centre-Back,Boro U18,-,in,Summer,NA,Premier League,1992,1992/1993
...,...,...,...,...,...,...,...,...,...,...,...,...
45,Sheffield Wednesday,Paul Williams,27,attack,Crystal Palace,?,out,Summer,NA,Premier League,1992,1992/1993
46,Wimbledon FC,Dean Holdsworth,23,Centre-Forward,Brentford,€1.00m,in,Summer,1,Premier League,1992,1992/1993
47,Wimbledon FC,Vinnie Jones,27,Defensive Midfield,Chelsea,€830Th.,in,Summer,0.83,Premier League,1992,1992/1993
48,Wimbledon FC,Greg Berry,21,midfield,Leyton Orient,€313Th.,in,Summer,0.313,Premier League,1992,1992/1993


In [4]:
%%ducksql

SELECT distinct year FROM staging.transfers_all
ORDER BY YEAR DESC

,year
0,2022
1,2021
2,2020
3,2019
4,2018
...,...
26,1996
27,1995
28,1994
29,1993


In [20]:
try:
    years = con.sql("SELECT DISTINCT CAST(year AS INT) y FROM staging.transfers_all WHERE year IS NOT NULL ORDER BY y").fetchall()
    source_view = "staging.transfers_all"
    year_col = "year"
except Exception:
    con.sql("""
        CREATE OR REPLACE VIEW staging.transfers_all_w_year AS
        SELECT *, CAST(strftime(try_cast(transfer_date AS DATE), '%Y') AS INT) AS yr
        FROM staging.transfers_all
    """)
    years = con.sql("SELECT DISTINCT yr AS y FROM staging.transfers_all_w_year WHERE yr IS NOT NULL ORDER BY y").fetchall()
    source_view = "staging.transfers_all_w_year"
    year_col = "yr"fssdfsdfsdf

con.sql("CREATE SCHEMA IF NOT EXISTS marts;")
for (y,) in years:
    con.sql(f"CREATE OR REPLACE TABLE marts.transfers_{y} AS SELECT * FROM {source_view} WHERE {year_col} = {y}")
    print(f"Created/updated marts.transfers_{y}")

Created/updated marts.transfers_1992
Created/updated marts.transfers_1993
Created/updated marts.transfers_1994
Created/updated marts.transfers_1995
Created/updated marts.transfers_1996
Created/updated marts.transfers_1997
Created/updated marts.transfers_1998
Created/updated marts.transfers_1999
Created/updated marts.transfers_2000
Created/updated marts.transfers_2001
Created/updated marts.transfers_2002
Created/updated marts.transfers_2003
Created/updated marts.transfers_2004
Created/updated marts.transfers_2005
Created/updated marts.transfers_2006
Created/updated marts.transfers_2007
Created/updated marts.transfers_2008
Created/updated marts.transfers_2009
Created/updated marts.transfers_2010
Created/updated marts.transfers_2011
Created/updated marts.transfers_2012
Created/updated marts.transfers_2013
Created/updated marts.transfers_2014
Created/updated marts.transfers_2015
Created/updated marts.transfers_2016
Created/updated marts.transfers_2017
Created/updated marts.transfers_2018
C

In [17]:
%%ducksql

/*
OBJ:
    - Add "Transfer Type" Column to the raw file.
*/


WITH src AS (
    SELECT
        fee
    FROM
        staging.transfers_all
)

SELECT
    fee,
    CASE
        WHEN LOWER(CAST(fee AS STRING)) LIKE '%loan%' THEN 'Loan'
        ELSE 'Transfer'
    END AS "Transfer Type"
FROM src
-- WHERE LOWER(CAST(fee AS STRING)) LIKE '%loan%';

,fee,Transfer Type
0,€910Th.,Transfer
1,€250Th.,Transfer
2,?,Transfer
3,-,Transfer
4,-,Transfer
...,...,...
23493,loan transfer,Loan
23494,loan transfer,Loan
23495,loan transfer,Loan
23496,"End of loanJun 30, 2022",Loan


In [47]:
# Get all column names in your table
cols = [c[0] for c in con.execute("DESCRIBE staging.transfers_all").fetchall()]

results = []
for c in cols:
    if c == 'fee':
        # Count cells with €
        euro_count = con.execute(f"""
            SELECT COUNT(fee) 
            FROM staging.transfers_all 
            WHERE CAST({c} AS STRING) LIKE '%€%'
        """).fetchone()[0]
        
        # Count cells with Loan
        loan_count = con.execute(f"""
            SELECT COUNT(fee) 
            FROM staging.transfers_all 
            WHERE CAST({c} AS STRING) LIKE LOWER('%Loan%')
        """).fetchone()[0]
        
        results.append((c, euro_count, loan_count))

import pandas as pd
df = pd.DataFrame(results, columns=["column", "count_euro", "count_loan"])

# 1) Columns with at least one €
df_euro = df[df["count_euro"] > 0]

# 2) Columns with at least one Loan
df_loan = df[df["count_loan"] > 0]

# 3) Columns with neither
df_neither = df[(df["count_euro"] == 0) & (df["count_loan"] == 0)]

display(df)         # full counts
display(df_euro)    # only euro
display(df_loan)    # only loan
display(df_neither) # neither

,column,count_euro,count_loan
0,fee,5701,11456


,column,count_euro,count_loan
0,fee,5701,11456


,column,count_euro,count_loan
0,fee,5701,11456


,column,count_euro,count_loan


In [5]:
%%ducksql


/*
OBJ:
- "free transfer" -> '0'
- '?' or '-' (but not euro/m/k/th cases) -> 'Unknown'
- values with 'm' -> replace 'm' with 000000, strip €, spaces, dots
- values with 'th' or 'k' -> replace with 000, strip €, spaces, dots
- everything else -> pass through (lowercased) so we can inspect as 'Dunno'


TURNS OUT THERE ALREADY A COLUMN CALLED "FEE CLEANED" FOR THIS. DIDN'T NEED TO DO ALL THIS LAST WEEK FFS

*/

WITH src AS (
  SELECT
    fee,
    LOWER(COALESCE(fee, '')) AS fee_l
  FROM staging.transfers_all
),
cleaned_data AS (
  SELECT
        fee_l,
    CASE
      WHEN fee_l LIKE '%free transfer%' THEN '0'

      WHEN (fee_l LIKE '%?%' OR fee_l LIKE '%-%')
           AND fee_l NOT LIKE '%th%'
           AND fee_l NOT LIKE '%k%'
           AND fee_l NOT LIKE '%m%'
           AND fee NOT LIKE '%€%' THEN 'Unknown'

      WHEN fee_l NOT LIKE '%loan%'
           AND fee LIKE '%€%'
           AND fee_l LIKE '%m%' THEN
           -- millions -> numeric text
           REGEXP_REPLACE(
             REGEXP_REPLACE(
               REGEXP_REPLACE(fee_l, 'm', '0000'),
               '[€\s]', ''
             ),
             '\.', ''
           )

      WHEN fee_l NOT LIKE '%loan%'
           AND fee LIKE '%€%'
           AND (fee_l LIKE '%th%' OR fee_l LIKE '%k%') THEN
           -- thousands ('th' or 'k') -> numeric text
           REGEXP_REPLACE(
             REGEXP_REPLACE(
               REGEXP_REPLACE(
                 REGEXP_REPLACE(fee_l, 'th', '000'),
                 'k', '000'
               ),
               '[€\s]', ''
             ),
             '\.', ''
           )

      WHEN fee_l LIKE '%?%' OR fee_l LIKE '%-%' THEN 'Unknown'

      ELSE fee_l
    END AS fee_cleaned,

    CASE 
      WHEN fee_l LIKE '%free transfer%' THEN 'Free Transfer'
      WHEN (fee_l LIKE '%?%' OR fee_l LIKE '%-%')
           AND fee_l NOT LIKE '%th%'
           AND fee_l NOT LIKE '%k%'
           AND fee_l NOT LIKE '%m%'
           AND fee NOT LIKE '%€%' THEN 'Unknown'
      WHEN fee_l NOT LIKE '%loan%' AND fee LIKE '%€%' AND fee_l LIKE '%m%' THEN 'Millions'
      WHEN fee_l NOT LIKE '%loan%' AND fee LIKE '%€%' AND (fee_l LIKE '%th%' OR fee_l LIKE '%k%') THEN 'Thousands'
      ELSE 'Dunno'
    END AS Reason
  FROM src
)

SELECT
    fee_l,
  fee_cleaned,
  Reason
FROM cleaned_data
-- WHERE Reason = 'Dunno';

,fee_l,fee_cleaned,Reason
0,€910th.,910000,Thousands
1,€250th.,250000,Thousands
2,?,Unknown,Unknown
3,-,Unknown,Unknown
4,-,Unknown,Unknown
...,...,...,...
23493,loan transfer,loan transfer,Dunno
23494,loan transfer,loan transfer,Dunno
23495,loan transfer,loan transfer,Dunno
23496,"end of loanjun 30, 2022","end of loanjun 30, 2022",Dunno


In [10]:
%%ducksql
/*
OBJ: filter and transform Fee values
     - keep only values with "€" in millions (exclude loans & thousands)
     - replace "m" with 000000
     - remove euro sign, spaces, and dots


-- CORRECT REPLACEMENT STRING:
    - REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(LOWER(fee), 'm', '0000'), '[€\s]', ''), '\.', '') AS millions_string

*/

WITH millions AS (
  SELECT DISTINCT fee
  FROM staging.transfers_all
  WHERE LOWER(fee) NOT LIKE '%loan%'
    AND fee LIKE '%€%'
    AND LOWER(fee) NOT LIKE '%th%' 
)

SELECT 
    fee,
    REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(LOWER(fee), 'm', '0000'), '[€\s]', ''), '\.', '') AS millions_string
FROM millions
LIMIT 50;

,fee,millions_string
0,€2.00m,2000000
1,€1.70m,1700000
2,€1.50m,1500000
3,€2.40m,2400000
4,€2.85m,2850000
...,...,...
45,€8.15m,8150000
46,€3.65m,3650000
47,€49.50m,49500000
48,€31.00m,31000000


In [9]:
%%ducksql

/*
I want to find a way to filter out only the values after the euro sign


  -- replace k with literal 0's for 1k <--- this is the one I want
  REGEXP_REPLACE(REGEXP_REPLACE(LOWER(fee), '[€.\s]', ''), 'th.', '000') AS stripped_th_in_thousands,

*/

WITH thousands AS (
  SELECT DISTINCT fee
  FROM staging.transfers_all
  WHERE LOWER(fee) NOT LIKE '%loan%'
    AND fee LIKE '%€%'
    AND LOWER(fee) LIKE '%th%'
)
SELECT
  fee,
  -- text after the euro sign, lowercased & trimmed
  TRIM(LOWER(SUBSTRING(fee, INSTR(fee, '€') + 1))) AS after_euro,

  -- remove euro, dots, and spaces
  REGEXP_REPLACE(LOWER(fee), '[€.\s]', '') AS no_euro_no_dot_no_space,

  -- also drop the literal 'th'
  REGEXP_REPLACE(REGEXP_REPLACE(LOWER(fee), '[€.\s]', ''), 'th.', 'k') AS stripped_th,

  -- replace k with literal 0's for 1k <--- this is the one I want
  REGEXP_REPLACE(REGEXP_REPLACE(LOWER(fee), '[€.\s]', ''), 'th.', '000') AS stripped_th_in_thousands,


FROM thousands
LIMIT 50;

,fee,after_euro,no_euro_no_dot_no_space,stripped_th,stripped_th_in_thousands
0,€112Th.,112th.,112th.,112k,112000
1,€94Th.,94th.,94th.,94k,94000
2,€65Th.,65th.,65th.,65k,65000
3,€70Th.,70th.,70th.,70k,70000
4,€500Th.,500th.,500th.,500k,500000
...,...,...,...,...,...
45,€318Th.,318th.,318th.,318k,318000
46,€745Th.,745th.,745th.,745k,745000
47,€88Th.,88th.,88th.,88k,88000
48,€57Th.,57th.,57th.,57k,57000


In [48]:
%%ducksql
SELECT
    COUNT(*) FILTER (WHERE LOWER(fee) LIKE '%loan%') AS loan_count,

    COUNT(*) FILTER (WHERE LOWER(fee) NOT LIKE '%loan%'
                     AND fee LIKE '%€%') AS euro_no_loan_count,

    COUNT(*) FILTER (WHERE LOWER(fee) NOT LIKE '%loan%'
                     AND fee NOT LIKE '%€%'
                     AND fee LIKE '%?%') AS questionmark_count,

    COUNT(*) FILTER (WHERE LOWER(fee) NOT LIKE '%loan%'
                     AND fee NOT LIKE '%€%'
                     AND LOWER(fee) LIKE '%free transfer%') AS free_transfer_count,

    COUNT(*) FILTER (WHERE LOWER(fee) NOT LIKE '%loan%'
                     AND fee NOT LIKE '%€%'
                     AND fee LIKE '%-%') AS dash_count
FROM staging.transfers_all;

,loan_count,euro_no_loan_count,questionmark_count,free_transfer_count,dash_count
0,11902,5255,1060,3271,2006


In [24]:
# Get all column names from your view
cols = [c[0] for c in con.execute("DESCRIBE staging.transfers_all").fetchall()]

# Build a query that counts nulls for each column
query = "SELECT\n" + ",\n".join(
    [f"COUNT(*) - COUNT({c}) AS {c}_nulls" for c in cols]
) + "\nFROM staging.transfers_all;"

df = con.sql(query).df()

# Transpose for easier reading: column vs. null count
df = df.T.reset_index()
df.columns = ["column_name", "null_count"]
display(df)

,column_name,null_count
0,club_name_nulls,0
1,player_name_nulls,0
2,age_nulls,0
3,position_nulls,0
4,club_involved_name_nulls,0
5,fee_nulls,4
6,transfer_movement_nulls,0
7,transfer_period_nulls,0
8,fee_cleaned_nulls,0
9,league_name_nulls,0
